In [8]:
import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
os.environ['CUDA_ENABLE_DEVICES'] = "1"
import pathlib
import tensorflow as tf

import time
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [9]:
PATH_TO_MODEL_DIR = "/home/dammeier@ab.ba.ba-ravensburg.de/dev/oil-storage-detection/data/06_models/models/efficientdet_d4"
PATH_TO_CONFIG = PATH_TO_MODEL_DIR + "/pipeline.config"
PATH_TO_CHECKPOINT = PATH_TO_MODEL_DIR

In [10]:
print('Loading model... ', end='')
start_time = time.time()

# Load pipeline confic and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CONFIG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
checkpoint = tf.compat.v2.train.Checkpoint(model=detection_model)
checkpoint.restore(os.path.join(PATH_TO_CHECKPOINT, 'ckpt-32')).expect_partial()

@tf.function
def detect_fn(image):
    """Detect objects in image."""
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    
    return detections

end_time = time.time()
elapsed_time = end_time - start_time
print(f'Done! Took {elapsed_time} seconds')

Loading model... Done! Took 18.651562690734863 seconds


In [11]:
category_index = {1: 'OST'}

In [12]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore') # suppress mpl warnings

def load_image_to_numpy_array(image_path):
    return np.array(Image.open(image_path))

def run_inference_on_image(image_path):
    print(f'Running inference for {image_path}...')
    
    image_np = load_image_to_numpy_array(image_path)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    
    detections = detect_fn(input_tensor)
    
    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=200,
            min_score_thresh=.30,
            agnostic_mode=False)

    plt.figure()
    plt.imshow(image_np_with_detections)
    print('Done')
    plt.show()
    

In [ ]:
run_inference_on_image('/home/dammeier@ab.ba.ba-ravensburg.de/dev/oil-storage-detection/data/02_intermediate/images/test/4fe78c6c-8c59-4f98-9083-f8117dc19ae0.jpg')

In [13]:
image_np = load_image_to_numpy_array('/home/dammeier@ab.ba.ba-ravensburg.de/dev/oil-storage-detection/data/02_intermediate/images/test/4fe78c6c-8c59-4f98-9083-f8117dc19ae0.jpg')
    
input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)

In [14]:
detections = detect_fn(input_tensor)

2022-05-24 15:29:58.799706: E tensorflow/stream_executor/cuda/cuda_blas.cc:232] failed to create cublas handle: CUBLAS_STATUS_ALLOC_FAILED
2022-05-24 15:29:58.800090: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at conv_ops.cc:1120 : NOT_FOUND: No algorithm worked!  Error messages:
  Profiling failure on CUDNN engine 1#TC: UNKNOWN: CUDNN_STATUS_EXECUTION_FAILED
in tensorflow/stream_executor/cuda/cuda_dnn.cc(4023): 'cudnnConvolutionForward( cudnn.handle(), alpha, input_nd_.handle(), input_data.opaque(), filter_.handle(), filter_data.opaque(), conv_.handle(), ToConvForwardAlgo(algo), scratch_memory.opaque(), scratch_memory.size(), beta, output_nd_.handle(), output_data.opaque())'
  Profiling failure on CUDNN engine 1: UNKNOWN: CUDNN_STATUS_EXECUTION_FAILED
in tensorflow/stream_executor/cuda/cuda_dnn.cc(4023): 'cudnnConvolutionForward( cudnn.handle(), alpha, input_nd_.handle(), input_data.opaque(), filter_.handle(), filter_data.opaque(), conv_.handle(), ToConvForwardA

NotFoundError: Graph execution error:

Detected at node 'EfficientDet-D4/model_1/stem_conv2d/Conv2D' defined at (most recent call last):
    File "/home/dammeier@ab.ba.ba-ravensburg.de/miniconda3/envs/tensorflow-osd/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/dammeier@ab.ba.ba-ravensburg.de/miniconda3/envs/tensorflow-osd/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/dammeier@ab.ba.ba-ravensburg.de/miniconda3/envs/tensorflow-osd/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/dammeier@ab.ba.ba-ravensburg.de/miniconda3/envs/tensorflow-osd/lib/python3.8/site-packages/traitlets/config/application.py", line 972, in launch_instance
      app.start()
    File "/home/dammeier@ab.ba.ba-ravensburg.de/miniconda3/envs/tensorflow-osd/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in star

In [ ]:
detections